In [69]:
from pyathena import connect
import pandas as pd
import boto3
from argparse import Namespace
import os
from sklearn.model_selection import train_test_split


In [71]:
args = Namespace(
    project_name = 'beer',
    aws_region = 'us-east-1',
    test_size = 0.25
)

In [8]:
conn = connect(s3_staging_dir=f's3://{args.project_name}-transformed',
               region_name='us-east-1')

df = pd.read_sql(f'SELECT * FROM "{args.project_name}-db"."{args.project_name}-table" limit 1000;', conn)
df.head()

,id,name,ibu,target_fg,target_og,ebc,srm,ph
0,128,White Noise,50.0,1012.0,1060.0,30.0,15.0,4.4
1,304,Karma Cloud,50.0,1010.0,1060.0,35.0,18.0,4.4
2,273,Prototype Double IPA,60.0,1007.0,1073.0,15.0,8.0,4.2
3,117,Hop Rocker,40.0,1010.0,1052.0,25.0,12.0,4.2
4,14,Alpha Dog,42.0,1025.0,1046.0,62.0,31.0,4.4


In [10]:
print(f'Total de dados {len(df)}')

458

In [47]:
len(df['name'].unique())

247

## Selecionar colunas

In [58]:
df = df[['ibu','target_fg','target_og','ebc','srm','ph']]

## Dados nulos

In [65]:
df.isna().sum()

ibu          0
target_fg    0
target_og    0
ebc          0
srm          0
ph           0
dtype: int64

In [63]:
df = df.dropna()

In [64]:
len(df)

448

## Separar dataset de treinamento e teste

In [30]:
def bucketExist(bucket_name) -> bool:
    s3 = boto3.client('s3')
    buckets = s3.list_buckets()
    return any(bucket['Name'] == bucket_name for bucket in buckets)

In [66]:
if not bucketExist(f'{args.project_name}-data-preparation'):
    s3 = boto3.client('s3')
    s3.create_bucket(Bucket=f'{args.project_name}-data-preparation')

In [73]:
df_train = df.sample(frac = args.test_size, random_state = 42)
df_test = df.drop(df_train.index)

In [74]:
len(df_train)

112

In [79]:
import s3fs

s3 = s3fs.S3FileSystem(anon=False)

# Use 'w' for py3, 'wb' for py2
with s3.open(f'{args.project_name}-data-preparation/train.csv','w') as f:
    df_train.to_csv(f)

In [80]:
with s3.open(f'{args.project_name}-data-preparation/test.csv','w') as f:
    df_test.to_csv(f)